In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import firm_revised

In [51]:
seed = 100

In [52]:
df = pd.read_csv(f'/Users/popovici/Desktop/Thesis/markdown_estimation/data/market_{seed}.csv')
print(df)

         price      cost  product  time      Car1      Car2    mshare  \
0     9.552859  1.845445        1     1  5.543405  2.810504  0.005380   
1     3.064592  1.761890        2     1  5.278369  3.255001  0.090384   
2     2.000000  2.000000        3     1  5.424518  2.541973  0.206187   
3     1.940064  1.940064        4     1  5.844776  3.435163  0.067304   
4    10.000000  1.616043        5     1  5.004719  2.416405  0.011396   
..         ...       ...      ...   ...       ...       ...       ...   
985   1.165361  1.165361        6    99  5.121569  3.816847  0.005733   
986   1.356730  1.356730        7    99  5.670749  3.672721  0.004404   
987   1.004820  1.004820        8    99  5.825853  2.895589  0.010860   
988   1.358923  1.358923        9    99  5.136707  2.468720  0.006016   
989   1.098656  1.098656       10    99  5.575093  4.029733  0.004738   

      profits   markups  
0    0.041466  4.176452  
1    0.117743  0.739378  
2    0.000000  0.000000  
3    0.000000  0.00

In [53]:
true_share = df['mshare']
print(true_share)

0      0.005380
1      0.090384
2      0.206187
3      0.067304
4      0.011396
         ...   
985    0.005733
986    0.004404
987    0.010860
988    0.006016
989    0.004738
Name: mshare, Length: 990, dtype: float64


In [54]:
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1


# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 500

# The number of period that I want to run the market for 
T = 100

In [67]:
# Unpacking the X and the price
X1 = np.array(df['Car1'][:J])
X2 = np.array(df['Car2'][:J])
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

eq_price = np.array(df['price'])

all_est_shares = np.zeros(J*(T-1))
print(all_est_shares.shape)

for t in range(0, T-1):
        v_p = np.random.normal(0, 1, size = N)
        e = np.random.gumbel(size=N*J)
        slice_price = eq_price[t*J:(t+1)*J] 
        print(slice_price)  
        est_share_period = firm_revised.share(N, J, X, v_p, slice_price, beta, mu, omega, e)
        all_est_shares[t*J:(t+1)*J] = est_share_period
        print(all_est_shares)



(990,)
[ 9.55285887  3.06459213  2.          1.94006388 10.          9.53608931
  1.79076326  2.          1.8375522   2.81331452]
[0.00898657 0.07195588 0.17942691 0.17433639 0.00950187 0.01494118
 0.13219182 0.1954848  0.15159479 0.05919387 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.

In [ ]:
# The idea is that you want to min the distance between the true share and the made up share by optimizing the fiven vector theta that determines the share 

def f(theta, true_share, N, J, X, price):
    beta = theta[:K+1]
    mu = theta[K+1]
    omega = theta[K+2]
    # make a numpy array all estimated shares 
    for t in range(0, T-1):
        v_p = np.random.normal(0, 1, size = N)
        e = np.random.gumbel(size=N*J)
        slice_price = eq_price[t*J:(t+1)*J] 
        print(slice_price)  
        est_share_period = firm_revised.share(N, J, X, v_p, slice_price, beta, mu, omega, e)
        all_est_shares[t*J:(t+1)*J] = est_share_period
        print(all_est_shares)
    return np.linalg.norm(true_share -all_est_shares)

In [ ]:
theta_0 = np.array([1., 1., 1., 1., 1.])

In [ ]:
res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
print(res.x)

[1 2 3]
4
5
